In [1]:
import pandas as pd
import numpy as np

In [2]:
### PREPROCESSING IMAGELAB COMMON LINES
ilab_df = pd.read_csv(r'..\databases\ILab_common_lines.txt', sep=";")
# Rename column to match final name
ilab_df = ilab_df.rename(columns={"Element" : "AtNum", "Intensity": "IntIlab"})
# Add source database
ilab_df["Source"] = "Ilab"
# Drop empty column
ilab_df.drop(["AtWeight"], axis=1, inplace=True)
# load element database -> used to harmonize columns with other databases
elements_db = pd.read_csv(r'..\databases\Periodic Table of Elements.csv')
elem_symbols = elements_db["Symbol"].tolist()
elem_atnum = elements_db["AtomicNumber"].tolist()
# Tranform ElemSymbol to AtNum
ilab_df["ElemSymbol"] = ilab_df["AtNum"].replace(elem_atnum, elem_symbols)
# Transform charge
ilab_df["Charge"] = ilab_df["Charge"] +1

In [3]:
### PREPROCESSING ATOMTRACE DATABASE
atomtrace_df = pd.read_csv(r'..\databases\database_AtomTrace_dropped_NaN_and_duplicates.csv', sep=",")
# Rename columns to match final name
atomtrace_df = atomtrace_df.rename(columns={"Element" : "ElemSymbol", "State" : "Charge"})
# Add source database
atomtrace_df["Source"] = "atomtrace"
# Replace charge roman numbers with numeric numbers
atomtrace_df["Charge"] = atomtrace_df["Charge"].replace(["I", "II"], [1, 2])
# Add AtNum
atomtrace_df["AtNum"] = atomtrace_df["ElemSymbol"].replace(elem_symbols, elem_atnum)
# Calculate intensity
atomtrace_df["IntAtomtrace"] = atomtrace_df["Aki"]*atomtrace_df["gk"]*np.exp((-atomtrace_df["Ek"]/((10**4)*(8.617330310*10**(-5)))))
# Drop uneccessary column
atomtrace_df.drop(["Identifier"], axis=1, inplace=True)

In [4]:
### CONCAT IMAGE LAB AND ATOMTRACE DATABASE
complete_db_df = pd.concat([ilab_df, atomtrace_df], ignore_index=True)[["ElemSymbol", "AtNum", "Charge", "Wavelength", "IntIlab", "Aki", "Ei", "Ek", "gi", "gk", "IntAtomtrace", "Source"]] # reorder columns as well
complete_db_df["Wavelength"] = complete_db_df["Wavelength"].round(decimals=2)
complete_db_df.to_csv(r'..\databases\common_Ilab_atomtrace_database_extended.csv')
complete_db_df

,ElemSymbol,AtNum,Charge,Wavelength,IntIlab,Aki,Ei,Ek,gi,gk,IntAtomtrace,Source
0,Xe,54,2,103.77,179.0,NaN,NaN,NaN,NaN,NaN,NaN,Ilab
1,Xe,54,2,104.13,230.0,NaN,NaN,NaN,NaN,NaN,NaN,Ilab
2,Ar,18,1,104.82,691.0,NaN,NaN,NaN,NaN,NaN,NaN,Ilab
3,Xe,54,2,104.83,208.0,NaN,NaN,NaN,NaN,NaN,NaN,Ilab
4,Xe,54,2,105.19,230.0,NaN,NaN,NaN,NaN,NaN,NaN,Ilab
...,...,...,...,...,...,...,...,...,...,...,...,...
2459,Th,90,2,438.19,NaN,11530000.0,0.831,3.659,10.0,10.0,1.651102e+06,atomtrace
2460,Th,90,2,439.11,NaN,6470000.0,0.557,3.379,6.0,8.0,1.025776e+06,atomtrace
2461,U,92,2,367.01,NaN,27709000.0,0.113,3.481,10.0,12.0,5.854027e+06,atomtrace
2462,U,92,2,409.01,NaN,12000000.0,0.217,3.248,14.0,14.0,3.876045e+06,atomtrace


In [5]:
complete_db_df_wo_duplicates = complete_db_df[["ElemSymbol", "AtNum", "Charge", "Wavelength"]].drop_duplicates(ignore_index=True)
complete_db_df_wo_duplicates.to_csv(r'..\databases\common_Ilab_atomtrace_database_dropped_duplicates.csv')
complete_db_df_wo_duplicates

,ElemSymbol,AtNum,Charge,Wavelength
0,Xe,54,2,103.77
1,Xe,54,2,104.13
2,Ar,18,1,104.82
3,Xe,54,2,104.83
4,Xe,54,2,105.19
...,...,...,...,...
2332,Tl,81,1,535.05
2333,Pb,82,1,368.37
2334,Pb,82,1,405.81
2335,Th,90,2,438.19
